# gh

> Command line utilities for github

In [ ]:
#| default_exp gh

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from fastcore.all import *
import itertools as it
import os
from ghapi.all import *

import configparser
from pathlib import Path

from oztools.core import *

from warnings import warn
import git
from git import Repo

import logging
logger = logging.getLogger(__name__)

To use this module you need to setup GitHub [personal access token](https://docs.github.com/github/authenticating-to-github/creating-a-personal-access-token), which is a secret code used to access your account.

If you don't have one, [click here](https://github.com/settings/tokens/new) to create one. You'll be asked to enter a name -- choose anything you like, for instance "oztools".
In "scopes" choose "repo", "gist", "notifications", and "workflow". Then clock "Generate Token" and copy the token. Then add it for example to your `.bashrc` as `export GITHUB_TOKEN="<your github token goes here>"`.

In [ ]:
warn("hello")

/tmp/ipykernel_1295329/502382720.py:1: UserWarning: hello
  warn("hello")


In [ ]:
#| export
def get_token_and_username():
    if "GITHUB_TOKEN" not in os.environ:
        warn("GITHUB_TOKEN is missing")
        return "GITHUB_TOKEN-missing", "username-missing"
    
    token = os.environ["GITHUB_TOKEN"]
    api = GhApi(token=token)
    if 'GITHUB_ACTOR' in os.environ:
        gh_username = "ozpau" # you can't get login name from github workflow
    else:
        gh_username = api.users.get_authenticated()['login']
    return token, gh_username

In [ ]:
#| export
token, gh_username = get_token_and_username()
api = GhApi(owner=gh_username, token=token)

In [ ]:
import oztools

In [ ]:
oztools?

Type:        module
String form: <module 'oztools' from '/home/mu/nbdev/oztools/oztools/__init__.py'>
File:        ~/nbdev/oztools/oztools/__init__.py
Docstring:   <no docstring>

In [ ]:
gh_username

'ozpau'

In [ ]:
repo = api.repos.get('make_llm')

In [ ]:
api.repos.get_latest_pages_build('make_llm')

HTTP404NotFoundError: HTTP Error 404: Not Found
====Error Body====
{
  "message": "Not Found",
  "documentation_url": "https://docs.github.com/rest/pages/pages#get-latest-pages-build",
  "status": "404"
}


In [ ]:
repo.branches_url

'https://api.github.com/repos/ozpau/make_llm/branches{/branch}'

In [ ]:
#| export
@call_parse
def gh_licenses():
    "List GitHub license templates"
    l = api.licenses.get_all_commonly_used(per_page=100)
    #return l.attrgot("spdx_id")
    return "\n".join(l.map(lambda x: f"{pad(x['spdx_id'],16)}{pad(x['name'],42)}"))

TODO: It would be nice if `@call_parse` created not just a cli wrapper, but also a raw function

In [ ]:
#| export
def gh_new_repo_fn(name, description, license, private):
    gh_repo = api.repos.create_for_authenticated_user(name, description, private=private, license_template=license)
    local_repo = Repo.clone_from(gh_repo.ssh_url, gh_repo.name)
    return gh_repo, local_repo

In [ ]:
#| export
@call_parse
def gh_new_repo(name:str, description:str, license:str="Apache-2.0", private:bool=False):
    "Create a new github repo and clone it"
    gh_new_repo_fn(name, description, license, private)

In [ ]:
#| export
def commit_and_push(repo: Repo, # Repo to commit and push
                    msg: str # Commit message
                   ):
    repo.git.add('.')
    repo.index.commit(msg)
    origin = repo.remote(name='origin')
    origin.push()

In [ ]:
#gh_new_repo("cadlab", "CAD tools for Jupyter notebooks")

In [ ]:
Path().cwd()

Path('/home/mu/nbdev/oztools/nbs/api')

In [ ]:
#from nbdev.cli import *

In [ ]:
#nbdev_new()

In [ ]:
#api.repos.list_for_org("")

In [ ]:
gh_repo = api.repos.get("ozpau.github.io")

In [ ]:
local_repo = Repo('../..')

In [ ]:
#head = local_repo.create_head('gh-pages')
#local_repo.git.push('--set-upstream', 'origin', head)

In [ ]:
#| export
def add_new_branch(repo: Repo, branch_name: str):
    "Create new branch and push it to upstream"
    head = repo.create_head(branch_name)
    repo.git.push('--set-upstream', 'origin', head)

Note that we need to split setting up branch in two parts
so that if we decide to commit something after setting up gh-pages
branch, it won't interrupt page deployment.

In [ ]:
#| export
def setup_pages_branch(local_repo: Repo, repo_name: str):
    add_new_branch(local_repo, 'gh-pages')

def setup_pages_branch_location(local_repo: Repo, repo_name: str):
    new_source = {"branch":"gh-pages"}
    try:
        api.repos.create_pages_site(repo_name, source={"branch":"gh-pages"})
    except HTTP409ConflictError:
        api.repos.update_information_about_pages_site(repo_name,
                                                      source={"branch":"gh-pages", "path": "/"})
    api.repos.request_pages_build(repo_name)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()